In [60]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib as mpl
import folium
from downloader import DownloadInfo
from sentinelhub import bbox_to_dimensions, BBox, CRS
import sklearn.cluster as skc
from sklearn.preprocessing import StandardScaler

%load_ext autoreload
%autoreload 2

mpl.rcParams['figure.dpi'] = 150

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
excel_df = pd.read_excel("converted_data.xlsx")

In [3]:
df = gpd.GeoDataFrame(
    excel_df, geometry=gpd.points_from_xy(excel_df["Northing"], excel_df["Easting"]), crs="EPSG:4326"
)
df.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [4]:
new_df = df.to_crs(epsg=3348)
new_df.crs

<Projected CRS: EPSG:3348>
Name: NAD83(CSRS) / Statistics Canada Lambert
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Canada - onshore and offshore - Alberta; British Columbia; Manitoba; New Brunswick; Newfoundland and Labrador; Northwest Territories; Nova Scotia; Nunavut; Ontario; Prince Edward Island; Quebec; Saskatchewan; Yukon.
- bounds: (-141.01, 38.21, -40.73, 86.46)
Coordinate Operation:
- name: Statistics Canada Lambert
- method: Lambert Conic Conformal (2SP)
Datum: NAD83 Canadian Spatial Reference System
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [5]:
polygon = gpd.GeoDataFrame(index=[0], crs=new_df.crs, geometry = [new_df.unary_union.convex_hull.buffer(1_000)])
polygon = polygon.to_crs(df.crs)

polygon.explore()

In [6]:

minx, miny, maxx, maxy = polygon.bounds["minx"][0], polygon.bounds["miny"][0], polygon.bounds["maxx"][0],polygon.bounds["maxy"][0]

m = folium.Map([maxy, maxx])

folium.Rectangle([[miny, minx], [maxy, maxx]], popup="Expanded bbox", color="blue", fill=True, fillopacity=0.2).add_to(m)

minx, miny, maxx, maxy = df.total_bounds[0], df.total_bounds[1], df.total_bounds[2], df.total_bounds[3]

folium.Rectangle([[miny, minx], [maxy, maxx]], popup="Original bbox", color="red", fill=True, fillopacity=0.2).add_to(m)

m

In [7]:
# Cluster wetlands that are close to each other

coordinates = new_df["geometry"].apply(lambda p: np.hstack(p.xy)).values
coordinates = np.vstack(coordinates)

In [90]:
scalar = StandardScaler()
scaled_features = scalar.fit_transform(coordinates)

def cluster(n_clusters):
    """Returns the dataframes, the bboxs, and then the k-means result (in that order)"""
    test_dfs = []
    bboxs = []
    kmeans = skc.KMeans(init="random", n_clusters=n_clusters, n_init=10)
    kmeans.fit(scaled_features)

    bboxs = []
    for i in range(n_clusters):
        matching_wetlands = []
        for j, wetland in new_df.iterrows():
            if kmeans.labels_[j] == i:
                matching_wetlands.append(wetland.geometry)
        test_geometry = gpd.GeoDataFrame(geometry=matching_wetlands, crs=new_df.crs).unary_union.convex_hull.buffer(1_000)
        test_df = gpd.GeoDataFrame(geometry = [test_geometry], crs=new_df.crs)
        test_df.to_crs(df.crs, inplace=True)
        bbox = BBox([test_df.bounds.minx.values[0], test_df.bounds.miny.values[0], test_df.bounds.maxx.values[0], test_df.bounds.maxy.values[0]], crs=CRS.WGS84)
        dimensions = bbox_to_dimensions(bbox, 10)
        if dimensions[0] > 2500 or dimensions[1] > 2500:
            print("dimensions are too big:", dimensions)
            return cluster(n_clusters + 1)
        else:
            bboxs.append(bbox)
            test_dfs.append(test_df)
    
    return test_dfs, bboxs, kmeans

result = cluster(3)
result[0]

dimensions are too big: (1228, 3711)


[                                            geometry
 0  POLYGON ((-111.71544 56.94102, -111.71627 56.9...,
                                             geometry
 0  POLYGON ((-111.33781 56.66411, -111.33746 56.6...,
                                             geometry
 0  POLYGON ((-111.56362 56.94382, -111.56508 56.9...,
                                             geometry
 0  POLYGON ((-111.28621 56.47527, -111.28622 56.4...]

In [88]:
download = DownloadInfo(result[1][0], 2019, 2019)
download.download()

This method will likely take a very long time to run


[1][11:33:02.739011] Data fetched for date 2019-05-17
[2][11:33:08.035382] Data fetched for date 2019-09-08
[3][11:33:13.564371] Data fetched for date 2019-06-22
[4][11:33:18.632838] Data fetched for date 2019-12-06
[5][11:33:23.970735] Data fetched for date 2019-06-04
[6][11:33:29.027749] Data fetched for date 2019-09-01
[7][11:33:34.238014] Data fetched for date 2019-12-01
[8][11:33:39.359544] Data fetched for date 2019-09-14
[9][11:33:44.272953] Data fetched for date 2019-01-16
[10][11:33:49.397405] Data fetched for date 2019-12-25
[11][11:33:55.950089] Data fetched for date 2019-03-12
[12][11:34:02.308458] Data fetched for date 2019-07-04
[13][11:34:07.413260] Data fetched for date 2019-10-02
[14][11:34:13.581706] Data fetched for date 2019-03-24
[15][11:34:18.870133] Data fetched for date 2019-03-17
[16][11:34:23.737806] Data fetched for date 2019-08-27
[17][11:34:40.063493] Data fetched for date 2019-05-23
[18][11:34:47.970679] Data fetched for date 2019-06-28
[19][11:34:56.36848

In [94]:
def get_color(i):
    if result[2].labels_[i] == 0:
        return "red"
    elif result[2].labels_[i] == 1:
        return "blue"
    elif result[2].labels_[i] == 2:
        return "green"
    elif result[2].labels_[i] == 3:
        return "purple"

m = folium.Map()

for i, wetland in df.iterrows():
    geometry = wetland["geometry"].xy
    folium.CircleMarker([geometry[1][0], geometry[0][0]], color=get_color(i), fill=True, fill_opacity=0.7, popup=wetland["Wetland"]).add_to(m)
m